# Mezclar Datos de Shopify

## Importar información

In [1]:
#importar librerias
import pandas as pd
import numpy as np
from enum import unique

In [2]:
# importar ordenes
orders = pd.read_csv("orders_export_1.csv")
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23682 entries, 0 to 23681
Data columns (total 74 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Name                         23682 non-null  object 
 1   Email                        23674 non-null  object 
 2   Financial Status             10528 non-null  object 
 3   Paid at                      9018 non-null   object 
 4   Fulfillment Status           10528 non-null  object 
 5   Fulfilled at                 10427 non-null  object 
 6   Accepts Marketing            10528 non-null  object 
 7   Currency                     10528 non-null  object 
 8   Subtotal                     10528 non-null  float64
 9   Shipping                     10528 non-null  float64
 10  Taxes                        10528 non-null  float64
 11  Total                        10528 non-null  float64
 12  Discount Code                5404 non-null   object 
 13  Discount Amount 

/var/folders/3n/2pry6xdj1_5_cqw2pyzdk21r0000gn/T/ipykernel_1028/551050056.py:2: DtypeWarning: Columns (20,33,43,44,45,46,71) have mixed types. Specify dtype option on import or set low_memory=False.
  orders = pd.read_csv("orders_export_1.csv")


In [3]:
#importar productos
products = pd.read_csv("products_export_1.csv")
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1895 entries, 0 to 1894
Data columns (total 49 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Handle                                     1895 non-null   object 
 1   Title                                      291 non-null    object 
 2   Body (HTML)                                237 non-null    object 
 3   Vendor                                     291 non-null    object 
 4   Standardized Product Type                  2 non-null      object 
 5   Custom Product Type                        217 non-null    object 
 6   Tags                                       287 non-null    object 
 7   Published                                  291 non-null    object 
 8   Option1 Name                               291 non-null    object 
 9   Option1 Value                              1657 non-null   object 
 10  Option2 Name            

In [4]:
#importar clientes
clients = pd.read_csv("customers_export_1.csv")
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18087 entries, 0 to 18086
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   First Name               16771 non-null  object 
 1   Last Name                16656 non-null  object 
 2   Email                    18079 non-null  object 
 3   Accepts Email Marketing  18087 non-null  object 
 4   Company                  1022 non-null   object 
 5   Address1                 16444 non-null  object 
 6   Address2                 3357 non-null   object 
 7   City                     16447 non-null  object 
 8   Province                 100 non-null    object 
 9   Province Code            100 non-null    object 
 10  Country                  16447 non-null  object 
 11  Country Code             16447 non-null  object 
 12  Zip                      16443 non-null  object 
 13  Phone                    8500 non-null   object 
 14  Accepts SMS Marketing 

In [5]:
#importar carritos abandonados
abandoned = pd.read_csv("checkouts_export_1.csv")
abandoned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1692 entries, 0 to 1691
Data columns (total 70 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Name                         1692 non-null   object 
 1   Email                        1692 non-null   object 
 2   Financial Status             0 non-null      float64
 3   Paid at                      0 non-null      float64
 4   Fulfillment Status           835 non-null    object 
 5   Fulfilled at                 0 non-null      float64
 6   Accepts Marketing            835 non-null    object 
 7   Currency                     835 non-null    object 
 8   Subtotal                     835 non-null    float64
 9   Shipping                     835 non-null    float64
 10  Taxes                        835 non-null    float64
 11  Total                        835 non-null    float64
 12  Discount Code                460 non-null    object 
 13  Discount Amount   

In [6]:
#Pegar ordenes y carrito abandonado

orders["source"] = "orders" #agregar columna para identificar la fuende de la información

abandoned["source"] = "carrito" #agregar columna para identificar la fuende de la información

orders_abandoned = pd.concat([orders,abandoned])

In [7]:
#limpieza de tabla de productos

products_dup = products.drop_duplicates(subset="Variant SKU",keep="first") #eliminar duplicados en la columna Variant SKU

products_clean = products_dup.dropna(subset=["Variant SKU"]) #eliminar las filas con NA en la columna Variant SKU

products_clean["Variant SKU"] = products_clean["Variant SKU"].apply(lambda x: str(x)[1:6]) #Dejar el SKU del producto cómo campo de texto, tomar los 5 caracteres despues del ', ej '02145


/var/folders/3n/2pry6xdj1_5_cqw2pyzdk21r0000gn/T/ipykernel_1028/2066501902.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_clean["Variant SKU"] = products_clean["Variant SKU"].apply(lambda x: str(x)[1:6]) #Dejar el SKU del producto cómo campo de texto, tomar los 5 caracteres despues del ', ej '02145


In [8]:
orders_abandoned["Lineitem sku"] = orders_abandoned["Lineitem sku"].apply(lambda x: str(x)[0:-2]) #Dejar el SKU del producto eliminando los 2 últimos caracteres.

#limpiar los SKU de la tablal orders_abandoned
temp = list()
for i in orders_abandoned["Lineitem sku"]:
    
    if len(i) == 4:
        temp.append('0'+i)
    elif len(i) == 3:
        temp.append('00'+i)
    elif len(i) == 2:
        temp.append('000'+i)
    elif len(i) == 0:
        temp.append('00000')
    else:
        temp.append(i)




In [9]:
orders_abandoned["Lineitem sku"] = temp

In [10]:

#cruzar las ordenes con carrito abandonado y los productos 

dataset = orders_abandoned.merge(products_clean, how= 'left', left_on="Lineitem sku",right_on="Variant SKU")
dataset.head()

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,...,Google Shopping / Custom Label 0,Google Shopping / Custom Label 1,Google Shopping / Custom Label 2,Google Shopping / Custom Label 3,Google Shopping / Custom Label 4,Variant Image,Variant Weight Unit,Variant Tax Code,Cost per item,Status
0,#83692,nicole.knaus@gmail.com,paid,2021-12-01 00:22:52 +0100,fulfilled,2021-12-01 13:22:23 +0100,no,EUR,70.2,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,kg,NaN,6.00,NaN
1,#83691,maudbertrand1994@gmail.com,paid,2021-11-30 22:59:01 +0100,fulfilled,2021-12-02 12:08:40 +0100,no,EUR,225.9,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,kg,NaN,9.43,NaN
2,#83691,maudbertrand1994@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,kg,NaN,6.00,NaN
3,#83691,maudbertrand1994@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,kg,NaN,8.88,NaN
4,#83691,maudbertrand1994@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,kg,NaN,9.43,NaN


## Juntando todo en una misma tabla







In [11]:
## Join dataset con customer

dataset = dataset.merge(clients, how='right',left_on='Email',right_on='Email')
#dataset.to_csv("dataset.csv", encoding='utf8')

dataset.head()


,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,...,Country,Country Code,Zip,Phone_y,Accepts SMS Marketing,Total Spent,Total Orders,Tags,Note,Tax Exempt
0,NaN,p.cheyssieu@free.fr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,France,FR,'38550,06 78 66 84 43,no,65.25,1,NaN,NaN,no
1,NaN,rosa.clara17@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,France,FR,'29820,06 59 46 32 18,no,83.30,1,#wgfr#wg,NaN,no
2,#32416670679274,shourm29@gmail.com,NaN,NaN,unfulfilled,NaN,no,EUR,33.15,4.9,...,France,FR,'29950,06 33 18 30 29,no,0.00,0,NaN,NaN,no
3,NaN,labarre.am@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,no,0.00,0,Privy,NaN,no
4,#32415980290282,loriagaelle@yahoo.fr,NaN,NaN,unfulfilled,NaN,no,EUR,61.20,4.9,...,France,FR,'82500,06 30 28 67 83,no,0.00,0,NaN,NaN,no


In [12]:

#nombre de las columnas que se quedarían

columnas = ['Email','First Name','Last Name','Accepts Email Marketing','Company','Address1','Address2','City','Province','Province Code',
            'Country','Country Code','Zip','Phone_y','Accepts SMS Marketing','Total Spent','Total Orders','Tags','Note','Name',
            'Financial Status','Paid at','Fulfillment Status','Fulfilled at','Currency','Subtotal','Shipping','Taxes','Total',
            'Discount Code','Discount Amount','Shipping Method','Created at','Lineitem quantity','Lineitem name','Lineitem price',
            'Lineitem compare at price','Lineitem sku','Lineitem requires shipping','Lineitem taxable','Lineitem fulfillment status',
            'Cancelled at','Payment Method','Payment Reference','Refunded Amount','Vendor_x','Id','Tags_x','Risk Level',
            'Source','Lineitem discount','Tax 1 Name','Tax 1 Value','Tax 2 Name','Tax 2 Value','Tax 3 Name','Tax 3 Value',
            'Tax 4 Name','Tax 4 Value','Tax 5 Name','Tax 5 Value','Phone_x','Receipt Number','Duties','source','Title',
            'Body (HTML)','Vendor_y','Standardized Product Type','Custom Product Type','Tags_y','Published','Option1 Name',
            'Option1 Value','Option2 Name','Option2 Value','Option3 Name','Option3 Value','Variant SKU','Variant Grams',
            'Variant Inventory Tracker','Variant Inventory Policy','Variant Fulfillment Service','Variant Price',
            'Variant Compare At Price','Variant Requires Shipping','Variant Taxable','Variant Barcode','Image Src',
            'Image Position','Image Alt Text','Gift Card','SEO Title','SEO Description','Google Shopping / Google Product Category',
            'Google Shopping / Gender','Google Shopping / Age Group','Google Shopping / MPN','Google Shopping / AdWords Grouping',
            'Google Shopping / AdWords Labels','Google Shopping / Condition','Google Shopping / Custom Product',
            'Google Shopping / Custom Label 0','Google Shopping / Custom Label 1','Google Shopping / Custom Label 2',
            'Google Shopping / Custom Label 3','Google Shopping / Custom Label 4','Variant Image','Variant Weight Unit',
            'Variant Tax Code','Cost per item','Status']
     
#Filtrar las columnas o variables

dataset_todo = dataset.filter(columnas,axis=1)

dataset_todo.to_csv("dataset_todo.csv", encoding='utf8')

dataset_todo.head()



,Email,First Name,Last Name,Accepts Email Marketing,Company,Address1,Address2,City,Province,Province Code,...,Google Shopping / Custom Label 0,Google Shopping / Custom Label 1,Google Shopping / Custom Label 2,Google Shopping / Custom Label 3,Google Shopping / Custom Label 4,Variant Image,Variant Weight Unit,Variant Tax Code,Cost per item,Status
0,p.cheyssieu@free.fr,Philippe,BEAL,no,NaN,445 Route Cuillery,NaN,Cheyssieu,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,rosa.clara17@gmail.com,Bodin,Fabienne,yes,NaN,27 rue de Kerguillo,NaN,Bohars,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,shourm29@gmail.com,Solenn,Hourmand,no,NaN,23 B Route De Nors Vras,NaN,Clohars-fouesnant,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,kg,NaN,7.83,NaN
3,labarre.am@gmail.com,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,loriagaelle@yahoo.fr,Gaelle,Loria,no,NaN,847 Chemin de Guinaud,NaN,Beaumont-de-Lomagne,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,kg,NaN,4.46,NaN


In [13]:
dataset_todo.describe()

,Total Spent,Total Orders,Subtotal,Shipping,Taxes,Total,Discount Amount,Lineitem quantity,Lineitem price,Lineitem compare at price,...,Google Shopping / Condition,Google Shopping / Custom Product,Google Shopping / Custom Label 0,Google Shopping / Custom Label 1,Google Shopping / Custom Label 2,Google Shopping / Custom Label 3,Google Shopping / Custom Label 4,Variant Image,Variant Tax Code,Cost per item
count,35550.000000,35550.000000,11179.000000,11179.000000,11179.000000,11179.000000,11179.000000,24984.000000,24984.000000,1250.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8830.000000
mean,207.240927,3.369480,73.774102,2.084199,11.941957,80.165272,11.739096,1.043868,37.021091,43.983360,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.039103
std,296.325088,13.620892,48.296853,2.909874,7.900002,48.889191,49.587749,0.275769,11.654766,14.646561,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.032158
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,15.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.950000
25%,53.420000,1.000000,39.750000,0.000000,7.320000,48.720000,0.000000,1.000000,32.040000,39.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.900000
50%,115.600000,2.000000,69.160000,0.000000,11.650000,75.000000,3.580000,1.000000,39.000000,43.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.960000
75%,251.220000,3.000000,94.000000,4.900000,15.330000,99.000000,9.500000,1.000000,43.000000,46.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.350000
max,5255.050000,513.000000,733.000000,19.900000,72.030000,733.000000,2473.000000,11.000000,200.000000,96.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.790000


In [14]:
#resumen de todas las variables para obtener los datos finales

dataset_final = dataset_todo.dropna(subset= ['Email'],axis=0)

labels = ['Email','First Name','Last Name','Accepts Email Marketing']


count_groups = ['Email','Name','Financial Status','Tags','Fulfillment Status','Shipping','Discount Code','Shipping Method','Payment Method',
                'Payment Reference','Risk Level','Source','Tax 1 Name','Tax 2 Name','Tax 3 Name','Tax 4 Name','Tax 5 Name',
                'Standardized Product Type','Custom Product Type','Tags_y','Published','Option1 Name',
                'Option1 Value','Option2 Name','Option2 Value','Option3 Name','Option3 Value','Variant SKU','Variant Grams',
                'Variant Inventory Tracker','Variant Inventory Policy','Variant Fulfillment Service','Variant Price',
                'Variant Compare At Price','Variant Requires Shipping','Variant Taxable','Variant Barcode',
                'Gift Card','Google Shopping / Google Product Category','Google Shopping / Gender',
                'Google Shopping / Age Group','Google Shopping / MPN','Google Shopping / AdWords Grouping',
                'Google Shopping / AdWords Labels','Google Shopping / Condition','Google Shopping / Custom Product',
                'Google Shopping / Custom Label 0','Google Shopping / Custom Label 1','Google Shopping / Custom Label 2',
                'Google Shopping / Custom Label 3','Google Shopping / Custom Label 4','Variant Image','Variant Weight Unit',
                'Variant Tax Code','Status']
                

transformations = {'Name':'nunique',
                    'Address1':'first',
                    'Address2':'first',
                    'City':'first',
                    'Province':'first',
                    'Province Code':'first',
                    'Country':'first',
                    'Country Code':'first',
                    'Zip':'first',
                    'Accepts SMS Marketing':'first',
                    'Subtotal':'sum',
                    'Taxes':'sum',
                    'Total':['sum','max'],
                    'Discount Amount':'sum',
                    'Lineitem quantity':'sum',
                    'Lineitem name':'nunique',
                    'Lineitem price':'mean',
                    'Lineitem taxable':'sum',
                    'Cancelled at':'first',
                    'Refunded Amount':'sum',
                    'Lineitem discount':'sum',
                    'Tax 1 Value':'sum',
                    'Tax 2 Value':'sum',
                    'Tax 3 Value':'sum',
                    'Tax 4 Value':'sum',
                    'Tax 5 Value':'sum',
                    'Cost per item':'sum'
                    }


col_names = ['total_orders','adress_1','address_2','city','province','province_code',
            'country','country_code','zip','accepts_marketing','subtotal','taxes','total',
            'mean_cart','discount_total','lineitem_quantity','n_references','lineitem_mean_price',
            'lineitem_taxable_references','canceled_at','refunded_total','lineitem_discount_total',
            'tax_1_total','tax_2_total','tax_3_total','tax_4_total','tax_5_total','cost_item_total']

dataset_final = dataset_final.groupby(by = labels,axis=0).agg(transformations)

dataset_final.columns = col_names

dataset_final = dataset_final.reset_index()

print(dataset_final.head())


                                    Email First Name         Last Name  \
0  0062f71d91f6202112@client.choose.rocks   Caroline            Mascle   
1  008c79d14d52202107@client.choose.rocks   Marjorie             BOEUF   
2  00cca15a3726202107@client.choose.rocks   Delphine            Cantet   
3  012659228e28202112@client.choose.rocks     Mareva   Perrier-bavoux    
4  020c36116bbf202112@client.choose.rocks     Claire               Bes   

  Accepts Email Marketing  total_orders                     adress_1  \
0                      no             0  16 Grande Rue Saint michel    
1                      no             0   19 avenue quakenbruck E12    
2                      no             0           5 rue des Vignoles   
3                      no             0                9 Rue Musette   
4                      no             0                  16 Calaudry   

                  address_2            city province province_code  ...  \
0                  Appt C12        Toulouse    

In [15]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder


categories_dataset = dataset_todo[count_groups]

#Identify the first row of every order
first_row = categories_dataset.duplicated(subset=['Name'],keep = 'first')

#Get the first row of the order
categories_dataset = categories_dataset.loc[first_row]

#replace all NA's by void text
categories_dataset = categories_dataset.fillna('')

categories_dataset.head()

#Let's encode all categories using OneHotEncoder
#This process needs to be done
one_hot = OneHotEncoder()

finantial_status = categories_dataset[['Financial Status']]


fin =one_hot.fit_transform(finantial_status)

fin.toarray()


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [16]:
one_hot.categories_

[array(['', 'paid'], dtype=object)]

In [17]:
dataset_final.shape

dataset_final.to_csv("dataset_limpio.csv", encoding='utf8',index = False)